In [1]:
import logging
import datetime
import json
import google.auth
import arrow
import json
import pandas as pd
import time
from pandas.io.json import json_normalize
import google.auth
from google.cloud import bigquery
from google.auth.transport.requests import AuthorizedSession
import warnings
warnings.filterwarnings('ignore')

def get_api(url):
    import google.auth
    credentials, project = google.auth.default()
    from google.auth.transport.requests import AuthorizedSession
    authed_session = AuthorizedSession(credentials)
    response = authed_session.get(url)
    return response

def post_api(url, data):
    import google.auth
    credentials, project = google.auth.default()
    from google.auth.transport.requests import AuthorizedSession
    authed_session = AuthorizedSession(credentials)
    response = authed_session.post(url, data)
    return response

def get_createdby(createTime, jobId):
    createdBy="NA"
    import arrow
    import json
    createTime = arrow.get(createTime)
    logstart=createTime + datetime.timedelta(minutes = -2)
    logend=createTime + datetime.timedelta(minutes = 2)
    body = {"resourceNames": ["projects/sb-bigdata-4985-da852265"], "filter":'resource.type="audited_resource" \
    resource.labels.service="ml.googleapis.com" \
    resource.labels.method="google.cloud.ml.v1.JobService.CreateJob" \
    timestamp>="' + str(logstart) + '" \
    timestamp<="' + str(logend) + '"'
    }
    y = json.dumps(body)
    response=post_api('https://logging.googleapis.com/v2/entries:list', y)
    import json
    import pandas as pd
    data_df = pd.read_json(response.text)
    if 'entries' in data_df:
        for job in data_df['entries']:
            if job["protoPayload"]["request"]["job"]["jobId"] == jobId:
                createdBy=job["protoPayload"]["authenticationInfo"]["principalEmail"]
                break
    else:
        if response.status_code == 429:
            import time
            # print("Sleeping 1 Sec: " + jobId)
            time.sleep(1)
            createdBy=get_createdby(createTime, jobId)
            #createdBy="Error"
            
    return createdBy

def get_labels_df(createTime, jobId):
    labels="NA"
    import arrow
    import pandas as pd
    createTime = arrow.get(createTime)
    logstart=createTime + datetime.timedelta(minutes = -2)
    logend=createTime + datetime.timedelta(minutes = 2)
    body = {"resourceNames": ["projects/sb-bigdata-4985-da852265"], "filter":'resource.type="audited_resource" \
    resource.labels.service="ml.googleapis.com" \
    resource.labels.method="google.cloud.ml.v1.JobService.CreateJob" \
    timestamp>="' + str(logstart) + '" \
    timestamp<="' + str(logend) + '"'
    }
    y = json.dumps(body)
    response=post_api('https://logging.googleapis.com/v2/entries:list', y)
    data_df = pd.read_json(response.text)
    if 'entries' in data_df:
        for job in data_df['entries']:
            if job["protoPayload"]["request"]["job"]["jobId"] == jobId and "labels" in job["protoPayload"]["request"]["job"]:
                labels=job["protoPayload"]["request"]["job"]["labels"]
                break
    else:
        if response.status_code == 429:
            import time
            # print("Sleeping 1 Sec: " + jobId)
            time.sleep(1)
            labels=get_labels_df(createTime, jobId)
    return labels

def get_cost(jobId):
    cost="NA"
    import json
    import pandas as pd
    response = get_api('https://ml.googleapis.com/v1/projects/sb-bigdata-4985-da852265/jobs/')
    data_df = pd.read_json(response.text)
    for i in range(len(data_df['jobs'])):
        if data_df['jobs'][i]['jobId'] == jobId:
            cost = 0.49 * (data_df['jobs'][i]['trainingOutput']['consumedMLUnits'])
            break
    return cost

def get_trainingStartTime(createTime, job_Id):
    trainingStartTime="NA"
    import arrow
    createTime = arrow.get(createTime)
    logstart=createTime + datetime.timedelta(minutes = -2)
    logend=createTime + datetime.timedelta(minutes = 2)
    body = {"resourceNames": ["projects/sb-bigdata-4985-da852265"], "filter":'resource.labels.task_name="master-replica-0" \
    Running task with arguments \
    timestamp>="' + str(logstart) + '" \
    timestamp<="' + str(logend) + '"'
    }
    y = json.dumps(body)
    response=post_api('https://logging.googleapis.com/v2/entries:list', y)
    import pandas as pd
    data_df = pd.read_json(response.text)
    if 'entries' in data_df:
        for i in range(len(data_df['entries'])):
            if data_df['entries'][i]['resource']['labels']['job_id'] == job_Id:
                st = data_df['entries'][i]['timestamp']
                # slice the timestamp to remove letters - T and Z. Also format the timestamp.
                sliceA = st[:10]
                startdate=datetime.datetime.strptime(sliceA, '%Y-%m-%d').strftime('%m%d%y')
                sliceB = st[11:26] 
                starttime=datetime.datetime.strptime(sliceB, '%H:%M:%S.%f').strftime('%H%M%S%f')
                # concatenate the sliced date and time segments
                concat1 = startdate + starttime
                # format the string of timestamp
                trainingStartTime_str=datetime.datetime.strptime(concat1, '%m%d%y%H%M%S%f').strftime('%Y-%m-%d %H:%M:%S.%f')
                # convert string to datetime object 
                trainingStartTime=datetime.datetime.strptime(trainingStartTime_str, '%Y-%m-%d %H:%M:%S.%f')
                break
    else:
        if response.status_code == 429:
            import time
            # print("Sleeping 1 Sec: " + jobId)
            time.sleep(1)
            trainingStartTime=get_trainingStartTime(createTime, job_Id)
    return trainingStartTime

# def get_trainingEndTime(createTime, job_Id):
#     trainingEndTime="NA"
#     import arrow
#     createTime = arrow.get(createTime)
#     logstart=createTime + datetime.timedelta(minutes = -2)
#     logend=createTime + datetime.timedelta(minutes = 2)
#     body = {"resourceNames": ["projects/sb-bigdata-4985-da852265"], "filter":'resource.labels.task_name="master-replica-0" \
#     Module completed cleaning up \
#     timestamp>="' + str(logstart) + '" \
#     timestamp<="' + str(logend) + '"'
#     }
#     y = json.dumps(body)
#     response=post_api('https://logging.googleapis.com/v2/entries:list', y)
#     import pandas as pd
#     data_df = pd.read_json(response.text)
#     if 'entries' in data_df:
#         for i in range(len(data_df['entries'])):
#             if data_df['entries'][i]['resource']['labels']['job_id'] == job_Id:
#                 et = data_df['entries'][i]['timestamp']
#                 # slice the timestamp to remove letters - T and Z. Also format the timestamp.
#                 sliceC = et[:10]
#                 enddate=datetime.datetime.strptime(sliceC, '%Y-%m-%d').strftime('%m%d%y')
#                 sliceD = et[11:26]
#                 endtime=datetime.datetime.strptime(sliceD, '%H:%M:%S.%f').strftime('%H%M%S%f')
#                 # concatenate the sliced date and time segments
#                 concat2 = enddate + endtime
#                 # format the string of timestamp
#                 trainingEndTime_str=datetime.datetime.strptime(concat2, '%m%d%y%H%M%S%f').strftime('%Y-%m-%d %H:%M:%S.%f')
#                 # convert string to datetime object 
#                 trainingEndTime=datetime.datetime.strptime(trainingEndTime_str, '%Y-%m-%d %H:%M:%S.%f')
#                 break   
#     else:
#         if response.status_code == 429:
#             import time
#             # print("Sleeping 1 Sec: " + jobId)
#             time.sleep(1)
#             trainingEndTime=get_trainingEndTime(createTime, job_Id)
#     return trainingEndTime

#resource.labels.job_id="census_training_1576841019" timestamp>="2019-12-20T11:23:43Z"
# resource.labels.job_id="sentiment_py3_basic_TPU_resolver_20191220_143926" 
# jsonPayload.message="Module completed; cleaning up."

def get_trainingEndTime(createTime, job_Id):
    #print("START get_trainingEndTime.....")
    trainingEndTime="NA"
    page_token = ''
    createTime = arrow.get(createTime)
  
    filter_str = 'resource.labels.job_id="' + str(job_Id) + '" \
    jsonPayload.message="Module completed; cleaning up."'
    body = {"resourceNames": ["projects/sb-bigdata-4985-da852265"],"pageToken" :str(page_token), "filter":filter_str }
    #print(body)
    y = json.dumps(body)
    response=post_api('https://logging.googleapis.com/v2/entries:list', y)
    #print(response.text)
    
    if response.status_code == 200:
        
        while True:
            data_df = pd.read_json(response.text, typ='series')
            #if 'entries' in data_df:
                #
                #for i in range(len(data_df['entries'])):
            if 'entries' in data_df and data_df['entries'][0]['resource']['labels']['job_id'] == job_Id:
                et = data_df['entries'][0]['timestamp']
                # slice the timestamp to remove letters - T and Z. Also format the timestamp.
                sliceC = et[:10]
                enddate=datetime.datetime.strptime(sliceC, '%Y-%m-%d').strftime('%m%d%y')
                sliceD = et[11:26]
                endtime=datetime.datetime.strptime(sliceD, '%H:%M:%S.%f').strftime('%H%M%S%f')
                # concatenate the sliced date and time segments
                concat2 = enddate + endtime
                # format the string of timestamp
                trainingEndTime_str=datetime.datetime.strptime(concat2, '%m%d%y%H%M%S%f').strftime('%Y-%m-%d %H:%M:%S.%f')
                # convert string to datetime object 
                trainingEndTime=datetime.datetime.strptime(trainingEndTime_str, '%Y-%m-%d %H:%M:%S.%f')
                #print("in main -->endtime for job_id {}. is {}.".format(job_Id ,trainingEndTime))
                break   
            #     
            else:
                #
                page_token = None
                page_token = data_df.get('nextPageToken')
                if page_token is None:
                    break
                else:
                    #
                    #print("****pagetoken recieved*****")
                    body = {"resourceNames": ["projects/sb-bigdata-4985-da852265"],"pageToken" :str(page_token), "filter":filter_str }
                    y = json.dumps(body)
                    response = post_api('https://logging.googleapis.com/v2/entries:list', y)

#    print("END get_trainingEndTime ... ")
    return trainingEndTime
    
def get_jobs_df():
    response = get_api('https://ml.googleapis.com/v1/projects/sb-bigdata-4985-da852265/jobs')
    job_list=[]
    i = 0
    if response.status_code == 200:
        import json
        import pandas as pd
        data_df = pd.read_json(response.text)
        #while True:
        for job in data_df['jobs']:
            if job['createTime'] >= max(job['createTime']):
                #job["trainingStartTime"]=get_trainingStartTime(job["createTime"], job["jobId"])
                job["trainingEndTime"]=get_trainingEndTime(job["createTime"], job["jobId"])
                job["createdby"]=get_createdby(job["createTime"], job["jobId"])
                job["labels"]=[get_labels_df(job["createTime"], job["jobId"])]
                job["trainingCost"]=get_cost(job["jobId"])
                job_list.append(job)
                i += 1
                print(i)
                page_token = None
                page_token = data_df.get('nextPageToken')
                if page_token is None:
                    break
                else:
                    response = get_api('https://ml.googleapis.com/v1/projects/sb-bigdata-4985-da852265/jobs' + '?pageToken=' + str(page_token[0]))
            else:
                None
    from pandas.io.json import json_normalize
    job_df=json_normalize(job_list)
    #print(job_df)
    job_df_new = job_df.rename(columns={"createTime": "createtime", "endTime": "endtime", "jobId":"jobid", "startTime":"starttime"
                       , "trainingInput.args":"args", "trainingInput.jobDir":"jobdir", "trainingInput.masterConfig.imageUri":"imageuri"
                       , "trainingInput.masterType":"mastertype", "trainingInput.packageUris":"packageuris", "trainingInput.pythonModule":"pythonmodule"
                       , "trainingInput.pythonVersion":"pythonversion", "trainingInput.region":"region", "trainingInput.runtimeVersion":"runtimeversion"
                       , "trainingInput.scaleTier":"scaletier", "trainingOutput.consumedMLUnits":"consumedmlunits", "labels":"label"
                       , "trainingInput.masterConfig.acceleratorConfig.count":"acceleratorconfigcount"
                       , "trainingInput.masterConfig.acceleratorConfig.type":"acceleratorconfigtype", "trainingCost":"trainingcost"
                       , "trainingEndTime":"trainingendtime"})
    return job_df_new

def write_to_bqtable(bq_tablename, bq_schemaname, datadf):
    import google.auth
    from google.cloud import bigquery
    credentials, project = google.auth.default()
    client = bigquery.Client(project)
    job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
    job_config.schema = bq_schemaname
    
    job = client.load_table_from_dataframe(datadf, bq_tablename, job_config=job_config)
    # Wait for the load job to complete.
    job.result()

def get_data(request):
    jobs = []
    jobs = get_jobs_df()
    jobs = jobs.drop(columns=["errorMessage", "etag"])
    print("AI platform jobs count : ",str(len(jobs)))
    jobs.insert(0, 'date', str(datetime.datetime.utcnow()))
    from google.cloud import bigquery
    varbqschema=[
        bigquery.SchemaField(name="date", field_type="STRING"),
        bigquery.SchemaField(name="createtime", field_type="STRING"),
        bigquery.SchemaField(name="createdby", field_type="STRING"),
        bigquery.SchemaField(name="endtime", field_type="STRING"),
        bigquery.SchemaField(name="jobid", field_type="STRING"),
        bigquery.SchemaField(name="starttime", field_type="STRING"),
        bigquery.SchemaField(name="state", field_type="STRING"),
        #bigquery.SchemaField(name="args", field_type="STRING"),
        bigquery.SchemaField(name="jobdir", field_type="STRING"),
        #bigquery.SchemaField(name="imageuri", field_type="STRING"),
        #bigquery.SchemaField(name="mastertype", field_type="STRING"),
        bigquery.SchemaField(name="packageuris", field_type="STRING"),
        bigquery.SchemaField(name="pythonmodule", field_type="STRING"),
        bigquery.SchemaField(name="pythonversion", field_type="STRING"),
        bigquery.SchemaField(name="region", field_type="STRING"),
        bigquery.SchemaField(name="runtimeversion", field_type="STRING"),
        bigquery.SchemaField(name="scaletier", field_type="STRING"),
        bigquery.SchemaField(name="consumedmlunits", field_type="STRING"),
        bigquery.SchemaField(name="label", field_type="STRING"),
        #bigquery.SchemaField(name="acceleratorconfigcount", field_type="STRING"),
        #bigquery.SchemaField(name="acceleratorconfigtype", field_type="STRING"),
        bigquery.SchemaField(name="trainingcost", field_type="STRING"),
        #bigquery.SchemaField(name="trainingstarttime", field_type="STRING"),
        bigquery.SchemaField(name="trainingendtime", field_type="STRING")
    ]
    write_to_bqtable('modelmanagement.jobmetrics', varbqschema, jobs.astype(str))
    return "Done!"

In [ ]:
#     remove_cols = pd.DataFrame(columns=['errorMessage', 'etag'])
#     for col in remove_cols.columns:
#         if col in jobs.columns:
#             jobs = jobs.drop(col, 1)
#         else:
#             None